In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

In [3]:
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Import URL content using _BeautifulSoup_

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [5]:
print(soup.prettify()[0:700])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6cde6ee6-c1f7-420d-8a4f-f54583ba3c90","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of po


Find all rows in the table and save to the variable rows

In [6]:
rows = soup.find_all("tr")
rows[0:5]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>, <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>, <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>]

Save the column names (header) to the list variable col

In [7]:
col = []
for r in rows[0].find_all("th"):
    col.append(r.string.strip('\n'))
print(col)

['Postal Code', 'Borough', 'Neighborhood']


Save the table contents in "rows" to the list variable in data

In [8]:
data = []
i = 1
for i in range(len(rows)):
    row = []
    for r in rows[i].find_all("td"):
            row.append(r.string)
    data.append(row)
    i+=1

In [9]:
del data[-4:]
del data[0]
print(data)

[['M1A\n', 'Not assigned\n', 'Not assigned\n'], ['M2A\n', 'Not assigned\n', 'Not assigned\n'], ['M3A\n', 'North York\n', 'Parkwoods\n'], ['M4A\n', 'North York\n', 'Victoria Village\n'], ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n'], ['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n'], ['M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n"], ['M8A\n', 'Not assigned\n', 'Not assigned\n'], ['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n'], ['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n'], ['M2B\n', 'Not assigned\n', 'Not assigned\n'], ['M3B\n', 'North York\n', 'Don Mills\n'], ['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n'], ['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n'], ['M6B\n', 'North York\n', 'Glencairn\n'], ['M7B\n', 'Not assigned\n', 'Not assigned\n'], ['M8B\n', 'Not assigned\n', 'Not assigned\n'], ['M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, 

Remove "\n" from the table contents and save to the new list variable ndata

In [10]:
ndata = []
for d in data:
    row = []
    for e in d:
        row.append(e.strip('\n'))
    ndata.append(row)
print(ndata)

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'], ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'], ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'], ['M1B', 'Scarborough', 'Malvern, Rouge'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills'], ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'], ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'], ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'], ['M2C', 'Not assigned', 'N

Create a dataframe with contents in ndata and column headers in col

In [11]:
df = pd.DataFrame(ndata, columns = col)
df.shape

(180, 3)

In [12]:
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Create a new dataframe toronto only containing rows needed for the project using the specified Postal Code 

In [14]:
toronto = pd.DataFrame(columns = col)
pc = ['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']
for i in range(len(pc)):
    toronto = toronto.append(df.loc[df['Postal Code'] == pc[i], :])
toronto

,Postal Code,Borough,Neighborhood
40,M5G,Downtown Toronto,Central Bay Street
46,M2H,North York,Hillcrest Village
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
54,M1J,Scarborough,Scarborough Village
39,M4G,East York,Leaside
84,M4M,East Toronto,Studio District
108,M1R,Scarborough,"Wexford, Maryvale"
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
80,M9L,North York,Humber Summit
139,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [15]:
toronto.reset_index(inplace=True)
toronto.drop(columns = 'index', inplace=True)
toronto

,Postal Code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


Import geospatial data that contains latitudes and longitudes of each neighborhood

In [16]:
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')
geospatial_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Create a dataframe coordinates that contains only the coordinates of the specified neighborhoods

In [17]:
coordinates = pd.DataFrame()
for i in range(len(pc)):
    coordinates = coordinates.append(geospatial_data.loc[geospatial_data['Postal Code'] == pc[i], :])
coordinates

,Postal Code,Latitude,Longitude
57,M5G,43.657952,-79.387383
17,M2H,43.803762,-79.363452
35,M4B,43.706397,-79.309937
5,M1J,43.744734,-79.239476
38,M4G,43.709060,-79.363452
43,M4M,43.659526,-79.340923
11,M1R,43.750072,-79.295849
101,M9V,43.739416,-79.588437
96,M9L,43.756303,-79.565963
68,M5V,43.628947,-79.394420


In [18]:
coordinates.reset_index(inplace=True)
coordinates.drop(columns = 'index', inplace=True)
coordinates

,Postal Code,Latitude,Longitude
0,M5G,43.657952,-79.387383
1,M2H,43.803762,-79.363452
2,M4B,43.706397,-79.309937
3,M1J,43.744734,-79.239476
4,M4G,43.709060,-79.363452
5,M4M,43.659526,-79.340923
6,M1R,43.750072,-79.295849
7,M9V,43.739416,-79.588437
8,M9L,43.756303,-79.565963
9,M5V,43.628947,-79.394420


Add latitude and longitude columns to the dataframe toronto

In [19]:
toronto['Latitude'] = coordinates['Latitude']
toronto['Longitude'] = coordinates['Longitude']
toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


Get the geographical coordinates of Toronto

In [20]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Visualize Toronto and the neighborhoods in it

In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [22]:
# The code was removed by Watson Studio for sharing.

CLIENT_ID, CLIENT_SECRET, and VERSION defined!


Create a function that generates URLs, make GET requests, import only relevent information for each venues and return a dataframe

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the getNearbyVenues function on each neighborhood and save to the dataframe toronto_venues

In [24]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                 latitudes=toronto['Latitude'],
                                 longitudes=toronto['Longitude']
                                 )

Central Bay Street
Hillcrest Village
Parkview Hill, Woodbine Gardens
Scarborough Village
Leaside
Studio District
Wexford, Maryvale
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Humber Summit
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Malvern, Rouge
Regent Park, Harbourfront


In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Bay Street,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
1,Central Bay Street,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant
2,Central Bay Street,43.657952,-79.387383,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop
3,Central Bay Street,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
4,Central Bay Street,43.657952,-79.387383,Panago,43.658258,-79.384313,Pizza Place


Venues count for each neighborhood

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,65,65,65,65,65,65
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2
Leaside,34,34,34,34,34,34
"Malvern, Rouge",1,1,1,1,1,1
"Parkview Hill, Woodbine Gardens",12,12,12,12,12,12
"Regent Park, Harbourfront",44,44,44,44,44,44
Scarborough Village,1,1,1,1,1,1


Number of unique venue categories

In [27]:
len(toronto_venues['Venue Category'].unique())

108

Create a new dataframe toronto_onehot with dummies for each venue category

In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(233, 108)

Make a new dataframe toronto_grouped with grouped rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.015385,0.000000,0.000000,0.000,0.000000,0.015385,0.030769,0.000000,0.015385,0.015385
2,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Leaside,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.088235,0.029412,0.000,0.029412,0.029412,0.000000,0.000000,0.000000,0.000000
5,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Parkview Hill, Woodbine Gardens",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Regent Park, Harbourfront",0.022727,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.022727,...,0.022727,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000
8,Scarborough Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
toronto_grouped.shape

(12, 108)

Write a function to sort the venue categories in descending order

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe neighborhoods_venues_sorted with top 10 venue categories for each neighborhood

In [37]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Rental Car Location,Boat or Ferry,Plane
1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Thai Restaurant,Bar,Bubble Tea Shop,Burger Joint
2,Hillcrest Village,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Athletics & Sports,Wine Bar,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop
3,Humber Summit,Pizza Place,Gym,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
4,Leaside,Coffee Shop,Sporting Goods Shop,Burger Joint,Bank,Furniture / Home Store,Dessert Shop,Department Store,Grocery Store,Gym,Bagel Shop
5,"Malvern, Rouge",Fast Food Restaurant,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store
6,"Parkview Hill, Woodbine Gardens",Pizza Place,Gastropub,Athletics & Sports,Pet Store,Pharmacy,Breakfast Spot,Bank,Café,Intersection,Gym / Fitness Center
7,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Bakery,Café,Theater,Restaurant,Breakfast Spot,Beer Store,Performing Arts Venue
8,Scarborough Village,Playground,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store
9,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Pharmacy,Beer Store,Cheese Shop,Chocolate Shop,Clothing Store


Run k-means clustering and cluster the neighborhoods into 5 clusters by venue categories

In [34]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 4, 3, 1, 0, 1, 1, 2, 1], dtype=int32)

Create a new dataframe by merging toronto and neighborhoods_venues_sorted and dropping postal codes column, which returns only boroughs, neighborhoods, coordinates for each neighborhood, newly-added cluster labels, and venue categories

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.drop(columns = 'Postal Code', inplace=True)
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Thai Restaurant,Bar,Bubble Tea Shop,Burger Joint
1,North York,Hillcrest Village,43.803762,-79.363452,4,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Athletics & Sports,Wine Bar,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop
2,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Gastropub,Athletics & Sports,Pet Store,Pharmacy,Breakfast Spot,Bank,Café,Intersection,Gym / Fitness Center
3,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store
4,East York,Leaside,43.709060,-79.363452,1,Coffee Shop,Sporting Goods Shop,Burger Joint,Bank,Furniture / Home Store,Dessert Shop,Department Store,Grocery Store,Gym,Bagel Shop
5,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Diner,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant
6,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,1,Auto Garage,Breakfast Spot,Middle Eastern Restaurant,Bakery,Sandwich Place,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
7,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,1,Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Pharmacy,Beer Store,Cheese Shop,Chocolate Shop,Clothing Store
8,North York,Humber Summit,43.756303,-79.565963,3,Pizza Place,Gym,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop
9,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Rental Car Location,Boat or Ferry,Plane


Create a map that visualizes the resulting clusters

In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine each cluster of venue categories

__Cluster 1__: mainly comprised of fast food restaurants, wine bars, diners, cheese, chocolate, coffee shops, etc.

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Malvern, Rouge",Fast Food Restaurant,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store


__Cluster 2__: mainly comprised of coffee shops or cafes, pubs, restaurants, bakeries, etc.

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Department Store,Thai Restaurant,Bar,Bubble Tea Shop,Burger Joint
2,"Parkview Hill, Woodbine Gardens",Pizza Place,Gastropub,Athletics & Sports,Pet Store,Pharmacy,Breakfast Spot,Bank,Café,Intersection,Gym / Fitness Center
4,Leaside,Coffee Shop,Sporting Goods Shop,Burger Joint,Bank,Furniture / Home Store,Dessert Shop,Department Store,Grocery Store,Gym,Bagel Shop
5,Studio District,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Diner,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant
6,"Wexford, Maryvale",Auto Garage,Breakfast Spot,Middle Eastern Restaurant,Bakery,Sandwich Place,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant
7,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Pharmacy,Beer Store,Cheese Shop,Chocolate Shop,Clothing Store
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Terminal,Rental Car Location,Boat or Ferry,Plane
11,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Bakery,Café,Theater,Restaurant,Breakfast Spot,Beer Store,Performing Arts Venue


__Cluster 3__: mainly comprised of playgrounds, wine bars, diners, cheese, chocolate, coffee shops, etc.

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough Village,Playground,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store


__Cluster 4__: mainly comprised of pizza places, gyms, wine bars, diners, etc.

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Humber Summit,Pizza Place,Gym,Wine Bar,Diner,Cheese Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop


__Cluster 5__: mainly comprised of golf courses, pools, Mediterranean restaurants, dog runs, etc.

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Hillcrest Village,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Athletics & Sports,Wine Bar,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop
